In [110]:
from datetime import datetime, timedelta
import pandas as pd
from pytimekr import pytimekr
import requests, json
from pandas import json_normalize


In [111]:
today = datetime.now()

In [112]:
today.strftime('%Y%m%y')

tyear = today.year
tmon = today.month
tday = today.day

tmon = 10
c_date = f'{tyear:02d}-{tmon:02d}-{tday:02d}'


ser_day = datetime.strptime(c_date, '%Y-%m-%d')


In [113]:
import calendar

# 0이면 월요일, 6이면 일요일이 시작일이다.
start_day_week, end_day = calendar.monthrange(tyear, tmon)
print(start_day_week, end_day)

# 해당 달의 첫날의 datetime형식으로 변환한다.
start_date = datetime.strptime(f'{tyear:02d}-{tmon:02d}-01', '%Y-%m-%d')
print(start_date)


6 31
2023-10-01 00:00:00


In [114]:
lines = []
before_month_day = 0
if start_day_week < 6:
    before_month_day = (start_date - timedelta(start_day_week+1)).day

    for i in range(start_day_week+1):
        lines.append(before_month_day)
        before_month_day +=1

print(lines)
    


[]


In [115]:
lines.extend(range(1, end_day+1))
print(lines)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [116]:
# 빈 칸에 다음 달 날짜 구하기
nday = calendar.weekday(tyear, tmon, end_day)
next_cnt = 0
if nday == 6: next_cnt = 6
elif nday < 5: next_cnt = 5 - nday

lines.extend(range(1, next_cnt+1))
print(lines)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 1, 2, 3, 4]


In [117]:
# 휴일 알아내기
pytimekr.holidays()

[datetime.date(2023, 1, 21),
 datetime.date(2023, 1, 22),
 datetime.date(2023, 1, 23),
 datetime.date(2023, 9, 28),
 datetime.date(2023, 9, 29),
 datetime.date(2023, 9, 30),
 datetime.date(2023, 1, 1),
 datetime.date(2023, 3, 1),
 datetime.date(2023, 5, 5),
 datetime.date(2023, 5, 26),
 datetime.date(2023, 6, 6),
 datetime.date(2023, 8, 15),
 datetime.date(2023, 10, 9),
 datetime.date(2023, 10, 3),
 datetime.date(2023, 12, 25)]

In [118]:
pytimekr.hangul()

datetime.date(2023, 10, 9)

In [119]:
with open('static/key/holidays.txt') as f:
    apikey = f.read()

- 공휴일

In [120]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey

In [121]:
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

print(res)

{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL SERVICE."},"body":{"items":{"item":[{"dateKind":"01","dateName":"1월1일","isHoliday":"Y","locdate":20230101,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230121,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230122,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230123,"seq":1},{"dateKind":"01","dateName":"대체공휴일","isHoliday":"Y","locdate":20230124,"seq":1},{"dateKind":"01","dateName":"삼일절","isHoliday":"Y","locdate":20230301,"seq":1},{"dateKind":"01","dateName":"어린이날","isHoliday":"Y","locdate":20230505,"seq":1},{"dateKind":"01","dateName":"부처님오신날","isHoliday":"Y","locdate":20230527,"seq":1},{"dateKind":"01","dateName":"대체공휴일","isHoliday":"Y","locdate":20230529,"seq":1},{"dateKind":"01","dateName":"현충일","isHoliday":"Y","locdate":20230606,"seq":1},{"dateKind":"01","dateName":"광복절","isHoliday":"Y","locdate":20230815,"seq":1},{"dateKind":"01","dateName":"추석","isHol

In [122]:
res = json.loads(res)
res.keys()

dict_keys(['response'])

In [123]:
res['response']['body']['items']['item']

[{'dateKind': '01',
  'dateName': '1월1일',
  'isHoliday': 'Y',
  'locdate': 20230101,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230121,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230122,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230123,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '대체공휴일',
  'isHoliday': 'Y',
  'locdate': 20230124,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '삼일절',
  'isHoliday': 'Y',
  'locdate': 20230301,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '어린이날',
  'isHoliday': 'Y',
  'locdate': 20230505,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '부처님오신날',
  'isHoliday': 'Y',
  'locdate': 20230527,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '대체공휴일',
  'isHoliday': 'Y',
  'locdate': 20230529,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '현충일',
  'isHoliday': 'Y',
  'locdate': 20230606,
  'seq': 1},
 {'dateKind': '01',
  'd

In [124]:
holidays = json_normalize(res['response']['body']['items']['item'])
holidays

,dateKind,dateName,isHoliday,locdate,seq
0,01,1월1일,Y,20230101,1
1,01,설날,Y,20230121,1
2,01,설날,Y,20230122,1
3,01,설날,Y,20230123,1
4,01,대체공휴일,Y,20230124,1
5,01,삼일절,Y,20230301,1
6,01,어린이날,Y,20230505,1
7,01,부처님오신날,Y,20230527,1
8,01,대체공휴일,Y,20230529,1
9,01,현충일,Y,20230606,1


- 24절기

In [125]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/get24DivisionsInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
res.keys()

dict_keys(['response'])

In [126]:
json_normalize(res['response']['body']['items']['item'])[['dateKind', 'dateName', 'isHoliday', 'locdate']]

,dateKind,dateName,isHoliday,locdate
0,03,소한,N,20230106
1,03,대한,N,20230120
2,03,입춘,N,20230204
3,03,우수,N,20230219
4,03,경칩,N,20230306
5,03,춘분,N,20230321
6,03,청명,N,20230405
7,03,곡우,N,20230420
8,03,입하,N,20230506
9,03,소만,N,20230521


- 기념일

In [127]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getAnniversaryInfo?_type=json&numOfRows=80&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
dt = pd.concat([holidays, json_normalize(res['response']['body']['items']['item'])])


In [128]:
dt

,dateKind,dateName,isHoliday,locdate,seq
0,01,1월1일,Y,20230101,1
1,01,설날,Y,20230121,1
2,01,설날,Y,20230122,1
3,01,설날,Y,20230123,1
4,01,대체공휴일,Y,20230124,1
...,...,...,...,...,...
64,02,아동학대예방의 날,N,20231119,1
65,02,소비자의 날,N,20231203,1
66,02,무역의 날,N,20231205,1
67,02,자원봉사자의 날,N,20231205,2


- 잡절


In [129]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getSundryDayInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
json_normalize(res['response']['body']['items']['item'])

,dateKind,dateName,isHoliday,locdate,seq
0,04,정월대보름,N,20230205,1
1,04,한식,N,20230406,1
2,04,단오,N,20230622,1
3,04,초복,N,20230711,2
4,04,중복,N,20230721,1
5,04,말복,N,20230810,1
6,04,칠석,N,20230822,1


- 한 번에 구하기

In [209]:
with open('static/key/holidays.txt') as f:
    apikey = f.read()
    
f_list = ['getRestDeInfo', 'getAnniversaryInfo', 'get24DivisionsInfo', 'getSundryDayInfo']
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/' #getSundryDayInfo?_type=json&numOfRows=50&solYear='
param1 = '?_type=json&numOfRows=80&solYear='
holidays = None
for name in f_list:
    url = base_url + name + param1 + str(tyear) + '&ServiceKey=' + apikey
    res = ''

    try:
        result = requests.get(url) 
        if result.status_code == 200:
            print('200')
            res = result.text
    except:
        pass

    if res: 
        res = json.loads(res)
        holidays = pd.concat([holidays, json_normalize(res['response']['body']['items']['item'])[['dateKind', 'dateName', 'isHoliday', 'locdate']] ])

200
200
200
200


In [210]:
holidays.head()

,dateKind,dateName,isHoliday,locdate
0,01,1월1일,Y,20230101
1,01,설날,Y,20230121
2,01,설날,Y,20230122
3,01,설날,Y,20230123
4,01,대체공휴일,Y,20230124


In [211]:
holidays.isna().sum().sum()

0

In [224]:
holidays['locdate'] = holidays['locdate'].astype(str)

In [225]:
type(list(holidays['locdate'])[0])

str

In [226]:
'20230101' in list(holidays['locdate'])

True

In [160]:
# holidays['locdate'].to_datetime()
holidays['locdate'] = pd.to_datetime(holidays['locdate'], format='%Y%m%d')

In [198]:
for dd in holidays['locdate'][:5]:
    print(dd.date().year, dd.date().month, dd.date().day)

2023 1 1
2023 1 21
2023 1 22
2023 1 23
2023 1 24


In [182]:
holidays

,dateKind,dateName,isHoliday,locdate
0,01,1월1일,Y,2023-01-01
1,01,설날,Y,2023-01-21
2,01,설날,Y,2023-01-22
3,01,설날,Y,2023-01-23
4,01,대체공휴일,Y,2023-01-24
...,...,...,...,...
2,04,단오,N,2023-06-22
3,04,초복,N,2023-07-11
4,04,중복,N,2023-07-21
5,04,말복,N,2023-08-10


In [203]:
pd.to_datetime('2023-01-01', format='%Y-%m-%d') in holidays['locdate']



False

In [206]:
for dd in holidays['locdate'][:5]:
    if dd == pd.to_datetime('2023-01-01', format='%Y-%m-%d'):
        print('OK')

OK
